In [1]:
#torchinfo summary (input(1, 3, 224, 224), verbose=2)

# separer les données en train et test et bien reppartir les classes ou ponderer par la frequence inverse ... WeightedRandomSampler ??
# Ajouter d'autre image avec transformation (X, Y, rotation ...)
# re train tous resnet et pas que la derniere couche
# Ajouter des workers à dataLoader
# utiliser tensorboard pour visualiser les données (utilse pour le compte rendu)

# rapport de 10-15 pages avec une date limite le 21/11 à 23h59.

# Le rapport doit détailler et analyser votre démarche, depuis le
# pré-traitement des données jusqu'aux évaluations de votre modèle en
# passant par la définition de l'architecture de celui-ci.
# - La présentation reprend le contenu du rapport de façon synthétique en
# plus d'autres contenus complémentaires si besoin.

#regarder la precision et le recall pour chaque classe
# prepare to count predictions for each class
    # correct_pred = {classname: 0 for classname in classes}
    # total_pred = {classname: 0 for classname in classes}

    # # again no gradients needed
    # with torch.no_grad():
    #     for data in testloader:
    #         images, labels = data
    #         outputs = net(images)
    #         _, predictions = torch.max(outputs, 1)
    #         # collect the correct predictions for each class
    #         for label, prediction in zip(labels, predictions):
    #             if label == prediction:
    #                 correct_pred[classes[label]] += 1
    #             total_pred[classes[label]] += 1


    # # print accuracy for each class
    # for classname, correct_count in correct_pred.items():
    #     accuracy = 100 * float(correct_count) / total_pred[classname]
    #     print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

In [2]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms, models
import torch.nn.functional as F

# Vérifier si CUDA est disponible, sinon utiliser le CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Architecture simplifiée du CNN
# class MelanomaCNN(nn.Module):
#     def __init__(self):
#         super(MelanomaCNN, self).__init__()
#         self.conv = nn.Conv2d(3, 16, kernel_size=3, padding=1)
#         self.fc = nn.Linear(16 * 112 * 112, 1)

#     def forward(self, x):
#         x = F.relu(self.conv(x))
#         x = F.max_pool2d(x, 2)
#         x = x.view(-1, 16 * 112 * 112)
#         x = torch.sigmoid(self.fc(x))
#         return x


# Ensemble de données personnalisé
class MelanomaDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None, limit=None):
        self.labels_df = pd.read_csv(csv_file).head(limit)  # Limite à 100 images
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.labels_df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.labels_df.iloc[idx, 0] + ".jpg")
        image = Image.open(img_name)
        label = torch.tensor(self.labels_df.iloc[idx, 1], dtype=torch.float32)
        if self.transform:
            image = self.transform(image)
        return image, label

In [3]:
# Transformations pour les images
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Charger l'ensemble de données
train_dataset = MelanomaDataset(csv_file='../../data/train-labels.csv',  # Ajustez le chemin
                                img_dir='../../data/train-resized/train-resized',      # Ajustez le chemin
                                transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=3) #workers ! 

# Charger ResNet50 pré-entraîné
model = models.resnet50(weights='ResNet50_Weights.IMAGENET1K_V2')

# Geler tous les paramètres du modèle
for param in model.parameters():
    param.requires_grad = False

# Remplacer la dernière couche fc pour notre classification binaire
model.fc = nn.Sequential(
    nn.Linear(2048, 1),
    nn.Sigmoid()
)

# Si une GPU est disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Fonction de perte et optimiseur
criterion = nn.BCELoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)  # Optimiser seulement les paramètres de model.fc
# Entraîner le modèle
epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        outputs = model(data).squeeze()
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(train_loader)}")


In [ ]:
from torchsummary import summary

summary(model, (1, 3, 224, 224))


NameError: name 'model' is not defined

In [ ]:
# Sauvegarder le modèle
torch.save(model.state_dict(), 'melanoma_model.pth')  # Ajustez le chemin
